## CLICKS
### clicks is at least 1

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file to explore its content and structure
campaign_data = pd.read_csv('Amazon Ads All.csv')

# Display the first few rows of the dataframe and the column names to understand the dataset
campaign_data.head(), campaign_data.columns


(   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
 0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
 1   66528  3003509  6064714310  offsite_mobileapp            4       0   
 2   88613  9302679  7428981046    offsite_desktop            4       0   
 3   57975  3325755  7489730099    offsite_desktop           13       0   
 4   13386  6532070  6004552148  offsite_mobileweb          246       0   
 
    auction_cost  adjusted_cost hit_day_utc  \
 0         10.89          8.790    02/05/24   
 1          0.08          0.032    02/05/24   
 2          0.08          0.032    02/05/24   
 3          0.26          0.127    02/05/24   
 4          2.46          1.978    02/05/24   
 
                             targeting_secondary                 vertical  \
 0  category=Automotive Replacement Control Arms               Automotive   
 1              category=Canned & Jarred Seafood  Consumer Packaged Goods   
 2                       category

In [2]:
print("Number of rows after deduplicating April 30 and May 1:", campaign_data.shape[0])

Number of rows after deduplicating April 30 and May 1: 566696


## CLEANING

In [3]:
## CLEANING
# Attempt to parse dates in the '%d/%m/%y' format
campaign_data['hit_day_utc_parsed'] = pd.to_datetime(campaign_data['hit_day_utc'], format='%d/%m/%y', errors='coerce')

# Parse remaining unparsed dates in the '%Y-%m-%d %H:%M:%S' format
mask = campaign_data['hit_day_utc_parsed'].isna()
campaign_data.loc[mask, 'hit_day_utc_parsed'] = pd.to_datetime(campaign_data.loc[mask, 'hit_day_utc'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check for any remaining non-parsed dates
remaining_non_parsed = campaign_data[campaign_data['hit_day_utc_parsed'].isna()]
if not remaining_non_parsed.empty:
    print("Remaining non-parsed dates:")
    print(remaining_non_parsed)

# Print unique months in the parsed 'hit_day_utc_parsed' column
print("Unique months in 'hit_day_utc_parsed':")
print(campaign_data['hit_day_utc_parsed'].dt.month.unique())

# Extract month and day of the week from parsed dates
campaign_data['month'] = campaign_data['hit_day_utc_parsed'].dt.month
campaign_data['day_of_week'] = campaign_data['hit_day_utc_parsed'].dt.dayofweek

campaign_data['month_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%B')  # Get month names
campaign_data['day_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%A')    # Get day names

# Fill missing values in 'vertical' and 'sub_vertical' with 'Unknown'
campaign_data['vertical'].fillna('Unknown', inplace=True)
campaign_data['sub_vertical'].fillna('Unknown', inplace=True)

# Identify the last 6 columns
last_6_columns = campaign_data.columns[-6:]

# Fill missing values in the last 6 columns with 0
campaign_data[last_6_columns] = campaign_data[last_6_columns].fillna(0)

# Display the cleaned dataset
print("Cleaned Campaign Data Sample:")
print(campaign_data.head())

# Inspect the 'hit_day_utc' column to understand its format
print("Sample of 'hit_day_utc' column:")
print(campaign_data['hit_day_utc'].head(10))

# Filter the data where clicks is at least 1
# campaign_data = campaign_data[campaign_data['clicks'] >= 1]


Unique months in 'hit_day_utc_parsed':
[5 4 3 2]


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_69669/749401865.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  campaign_data['vertical'].fillna('Unknown', inplace=True)
/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_69669/749401865.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting va

Cleaned Campaign Data Sample:
   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
1   66528  3003509  6064714310  offsite_mobileapp            4       0   
2   88613  9302679  7428981046    offsite_desktop            4       0   
3   57975  3325755  7489730099    offsite_desktop           13       0   
4   13386  6532070  6004552148  offsite_mobileweb          246       0   

   auction_cost  adjusted_cost hit_day_utc  \
0         10.89          8.790    02/05/24   
1          0.08          0.032    02/05/24   
2          0.08          0.032    02/05/24   
3          0.26          0.127    02/05/24   
4          2.46          1.978    02/05/24   

                            targeting_secondary  ...  \
0  category=Automotive Replacement Control Arms  ...   
1              category=Canned & Jarred Seafood  ...   
2                       category=Comedy & Humor  ...   
3                  categ

In [4]:
print("Number of rows after deduplicating April 30 and May 1:", campaign_data.shape[0])
campaign_data = campaign_data[campaign_data['clicks'] >= 1]
print("Number of rows after deduplicating April 30 and May 1:", campaign_data.shape[0])

Number of rows after deduplicating April 30 and May 1: 566696
Number of rows after deduplicating April 30 and May 1: 58674


## Model 1

In [11]:
# Prepare the dataset

# Display columns to ensure the correct ones are present
print("Columns in Campaign Data:")
print(campaign_data.columns)

# Check if 'placement_slot', 'targeting_secondary', 'vertical', and 'sub_vertical' exist in the columns before encoding
categorical_columns = ['placement_slot', 'targeting_secondary', 'vertical', 'sub_vertical', 'month_name', 'day_name']
existing_categorical_columns = [col for col in categorical_columns if col in campaign_data.columns]

if existing_categorical_columns:
    campaign_data = pd.get_dummies(campaign_data, columns=existing_categorical_columns, drop_first=True)

# Select features and target variable
features = ['impressions'] + \
           [col for col in campaign_data.columns if col.startswith('placement_slot_')] + \
           [col for col in campaign_data.columns if col.startswith('targeting_secondary_')] + \
           [col for col in campaign_data.columns if col.startswith('vertical_')] + \
           [col for col in campaign_data.columns if col.startswith('sub_vertical_')] + \
           [col for col in campaign_data.columns if col.startswith('month_name_')] + \
           [col for col in campaign_data.columns if col.startswith('day_name_')]

target = 'clicks'

X = campaign_data[features]
y = campaign_data[target]

# Handle missing values using SimpleImputer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

if len(numeric_features) > 0:
    numeric_imputer = SimpleImputer(strategy='mean')
    X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])

if len(categorical_features) > 0:
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])

# Check if any missing values remain
print("Missing values in X after imputation:")
print(X.isna().sum().sum())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train, y_train)

# Predict on the test set
y_pred = gbr.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the evaluation metrics
evaluation_metrics = {
    'Mean Absolute Error': mae,
    'Mean Squared Error': mse,
    'R-squared': r2
}

print("Evaluation Metrics:")
print(pd.DataFrame([evaluation_metrics]))

# Display feature importances
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': gbr.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance for Gradient Boosting Regressor:")
pd.set_option('display.max_rows', None)  # Ensure all rows are displayed
print(feature_importance_df)

Columns in Campaign Data:
Index(['ADV_ID', 'AD_ID', 'SKU', 'placement_slot', 'impressions', 'clicks',
       'auction_cost', 'adjusted_cost', 'hit_day_utc', 'targeting_secondary',
       'vertical', 'sub_vertical', 'ntb_click_attributed_units_sold',
       'ntb_view_attributed_units_sold', 'ntb_click_attributed_orders',
       'ntb_view_attributed_orders', 'view_attributed_units_sold',
       'view_attributed_orders', 'hit_day_utc_parsed', 'month', 'day_of_week',
       'month_name', 'day_name'],
      dtype='object')


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_65623/1391019023.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])


Missing values in X after imputation:
0
Evaluation Metrics:
   Mean Absolute Error  Mean Squared Error  R-squared
0             1.230959            4.073486   0.736022

Feature Importance for Gradient Boosting Regressor:
                                                Feature  Importance
2                      placement_slot_offsite_mobileweb    0.490113
0                                           impressions    0.397149
1067                                month_name_February    0.027180
788   targeting_secondary_category=Sheet & Pillowcas...    0.012234
802   targeting_secondary_category=Single-Serve Coff...    0.010674
713   targeting_secondary_category=Pipe Fittings & P...    0.009292
398         targeting_secondary_category=Golf Equipment    0.006984
1033                                 vertical_Softlines    0.006481
88    targeting_secondary_category=Bed Pillows & Pos...    0.004980
1071                                  day_name_Saturday    0.003218
1043                        sub

In [12]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(feature_importance_df)

                                                                             Feature  \
2                                                   placement_slot_offsite_mobileweb   
0                                                                        impressions   
1067                                                             month_name_February   
788                             targeting_secondary_category=Sheet & Pillowcase Sets   
802                 targeting_secondary_category=Single-Serve Coffee Capsules & Pods   
713                               targeting_secondary_category=Pipe Fittings & Pipes   
398                                      targeting_secondary_category=Golf Equipment   
1033                                                              vertical_Softlines   
88                            targeting_secondary_category=Bed Pillows & Positioners   
1071                                                               day_name_Saturday   
1043                            

## Model 2

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Filter
campaign_data = campaign_data[campaign_data['clicks'] >= 1]

# Define features and target for adjusted cost
features_cost = ['impressions', 'targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical']
target_cost = 'clicks'

# Prepare the data
X_cost = campaign_data[features_cost]
y_cost = campaign_data[target_cost]

# One-hot encode categorical features and standardize numerical features
preprocessor_cost = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [ 'impressions']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical'])
    ])

# Define and train 
model_cost = Pipeline(steps=[
    ('preprocessor', preprocessor_cost),
    ('regressor', RandomForestRegressor(random_state=42))
])

X_train_cost, X_test_cost, y_train_cost, y_test_cost = train_test_split(X_cost, y_cost, test_size=0.2, random_state=42)
model_cost.fit(X_train_cost, y_train_cost)

# Predict and evaluate the adjusted cost model
y_pred_cost = model_cost.predict(X_test_cost)
mse_cost = mean_squared_error(y_test_cost, y_pred_cost)
mae_cost = mean_absolute_error(y_test_cost, y_pred_cost)
r2_cost = r2_score(y_test_cost, y_pred_cost)

# Display the evaluation metrics for adjusted cost model
print(f"Adjusted Cost Model - Mean Squared Error: {mse_cost}")
print(f"Adjusted Cost Model - Mean Absolute Error: {mae_cost}")
print(f"Adjusted Cost Model - R-squared: {r2_cost}")

# Get feature names after one-hot encoding
encoded_feature_names_cost = model_cost.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(['targeting_secondary', 'placement_slot', 'month_name', 'day_name',
                                                                                                               'vertical', 'sub_vertical'])

# Combine original numerical features with encoded categorical features
all_features_cost = ['impressions'] + list(encoded_feature_names_cost)

# Calculate feature importance
importance_cost = model_cost.named_steps['regressor'].feature_importances_
feature_importance_cost = pd.DataFrame({
    'Feature': all_features_cost,
    'Importance': importance_cost
}).sort_values(by='Importance', ascending=False)

print("Feature Importance for Adjusted Cost Model:")
pd.set_option('display.max_rows', None)
print(feature_importance_cost)

Adjusted Cost Model - Mean Squared Error: 0.7704317211568181
Adjusted Cost Model - Mean Absolute Error: 0.24494389560138635
Adjusted Cost Model - R-squared: 0.9500730591649822
Feature Importance for Adjusted Cost Model:
                                                Feature    Importance
0                                           impressions  4.220244e-01
973                    placement_slot_offsite_mobileweb  1.838726e-01
971                      placement_slot_offsite_desktop  1.653110e-01
975                                 month_name_February  2.486699e-02
82    targeting_secondary_category=Bed Pillows & Pos...  1.046177e-02
980                                   day_name_Saturday  1.036627e-02
981                                     day_name_Sunday  1.036171e-02
755   targeting_secondary_category=Single-Serve Coff...  1.008665e-02
979                                     day_name_Monday  8.517763e-03
376         targeting_secondary_category=Golf Equipment  8.201275e-03
742   targ

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Filter
campaign_data = campaign_data[campaign_data['clicks'] >= 1]

# Define features and target for adjusted cost
features_cost = ['impressions', 'targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical']
target_cost = 'clicks'

# Prepare the data
X_cost = campaign_data[features_cost]
y_cost = campaign_data[target_cost]

# One-hot encode categorical features and standardize numerical features
preprocessor_cost = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [ 'impressions']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical'])
    ])

# Define and train 
model_cost = Pipeline(steps=[
    ('preprocessor', preprocessor_cost),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

X_train_cost, X_test_cost, y_train_cost, y_test_cost = train_test_split(X_cost, y_cost, test_size=0.2, random_state=42)
model_cost.fit(X_train_cost, y_train_cost)

# Predict and evaluate the adjusted cost model
y_pred_cost = model_cost.predict(X_test_cost)
mse_cost = mean_squared_error(y_test_cost, y_pred_cost)
mae_cost = mean_absolute_error(y_test_cost, y_pred_cost)
r2_cost = r2_score(y_test_cost, y_pred_cost)

# Display the evaluation metrics for adjusted cost model
print(f"Adjusted Cost Model - Mean Squared Error: {mse_cost}")
print(f"Adjusted Cost Model - Mean Absolute Error: {mae_cost}")
print(f"Adjusted Cost Model - R-squared: {r2_cost}")

# Get feature names after one-hot encoding
encoded_feature_names_cost = model_cost.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(['targeting_secondary', 'placement_slot', 'month_name', 'day_name',
                                                                                                               'vertical', 'sub_vertical'])

# Combine original numerical features with encoded categorical features
all_features_cost = ['impressions'] + list(encoded_feature_names_cost)

# Calculate feature importance
importance_cost = model_cost.named_steps['regressor'].feature_importances_
feature_importance_cost = pd.DataFrame({
    'Feature': all_features_cost,
    'Importance': importance_cost
}).sort_values(by='Importance', ascending=False)

print("Feature Importance for Adjusted Cost Model:")
pd.set_option('display.max_rows', None)
print(feature_importance_cost)

Adjusted Cost Model - Mean Squared Error: 3.9885837120656613
Adjusted Cost Model - Mean Absolute Error: 1.224310401370135
Adjusted Cost Model - R-squared: 0.74152442385315
Feature Importance for Adjusted Cost Model:
                                                Feature  Importance
0                                           impressions    0.400810
971                      placement_slot_offsite_desktop    0.245799
973                    placement_slot_offsite_mobileweb    0.241649
975                                 month_name_February    0.026527
742   targeting_secondary_category=Sheet & Pillowcas...    0.011744
755   targeting_secondary_category=Single-Serve Coff...    0.010699
673   targeting_secondary_category=Pipe Fittings & P...    0.009220
376         targeting_secondary_category=Golf Equipment    0.008482
990                                  vertical_Softlines    0.006572
82    targeting_secondary_category=Bed Pillows & Pos...    0.004853
1001                        sub_vert